In [13]:
import numpy as np
import os
import torch

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50
from tqdm import tqdm
from typing import Dict

from l5kit.geometry import transform_points
from l5kit.evaluation import write_pred_csv
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from IPython.display import HTML
#from IPython.display import display, clear_output
import PIL
import numpy as np
import pandas as pd
import scipy as sp
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable
from IPython.display import display, clear_output
import os
from scipy import stats
from pathlib import Path
import zarr
import itertools as it
import sys

In [14]:
os.environ["L5KIT_DATA_FOLDER"] = "data"
# cfg = load_config_data("examples/visualisation/visualisation_config.yaml")


cfg = {
    'format_version': 4,
    'model_params': {
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 12,
        'shuffle': False,
        'num_workers': 0
    }

}


# print(cfg)
dm = LocalDataManager()
dataset_path = dm.require('scenes/sample.zarr')
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()

frames = zarr_dataset.frames
agents = zarr_dataset.agents
scenes = zarr_dataset.scenes
tl_faces = zarr_dataset.tl_faces

display(str(frames))
display(str(agents))
display(str(scenes))
display(str(tl_faces))

"<zarr.core.Array '/frames' (24838,) [('timestamp', '<i8'), ('agent_index_interval', '<i8', (2,)), ('traffic_light_faces_index_interval', '<i8', (2,)), ('ego_translation', '<f8', (3,)), ('ego_rotation', '<f8', (3, 3))] read-only>"

"<zarr.core.Array '/agents' (1893736,) [('centroid', '<f8', (2,)), ('extent', '<f4', (3,)), ('yaw', '<f4'), ('velocity', '<f4', (2,)), ('track_id', '<u8'), ('label_probabilities', '<f4', (17,))] read-only>"

"<zarr.core.Array '/scenes' (100,) [('frame_index_interval', '<i8', (2,)), ('host', '<U16'), ('start_time', '<i8'), ('end_time', '<i8')] read-only>"

"<zarr.core.Array '/traffic_light_faces' (316008,) [('face_id', '<U16'), ('traffic_light_id', '<U16'), ('traffic_light_face_status', '<f4', (3,))] read-only>"

In [15]:
def build_model(cfg: Dict) -> torch.nn.Module:
    # load pre-trained Conv2D model
    model = resnet50(pretrained=False)

    # change input channels number to match the rasterizer's output
    num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
    num_in_channels = 3 + num_history_channels
    model.conv1 = nn.Conv2d(
        num_in_channels,
        model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding,
        bias=False,
    )
    # change output size to (X, Y) * number of future states
    num_targets = 2 * cfg["model_params"]["future_num_frames"]
    model.fc = nn.Linear(in_features=2048, out_features=num_targets)

    return model

def forward(data, model, device):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    outputs = model(inputs).reshape(targets.shape)
    return outputs


In [16]:
test_cfg = cfg["test_data_loader"]

test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load("data/scenes/mask.npz")["arr_0"]

rasterizer = build_rasterizer(cfg, dm)
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])
print(test_dataset)


C:\Users\Omar\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: you're running with a custom agents_mask
  import sys


+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   11314    |  1131400   |  88594921  |    7854144    |      31.43      |        100.00        |        78.31         |        10.00         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [ ]:
# ==== INIT MODEL
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = build_model(cfg).to(device)

model.load_state_dict(torch.load("baseline_weights/baseline_weights.pth", map_location=device))
# ==== EVAL LOOP
model.eval()
torch.set_grad_enabled(False)

# store information for evaluation
future_coords_offsets_pd = []
timestamps = []
agent_ids = []

progress_bar = tqdm(test_dataloader)
for data in progress_bar:
    outputs = forward(data, model, device).cpu().numpy().copy()

    # convert into world coordinates offsets
    coords_offset = []
    for agent_coords, agent_yaw_rad in zip(outputs, data["yaw"]):
        world_offset_from_agent = np.array(
        [
            [np.cos(agent_yaw_rad), -np.sin(agent_yaw_rad), 0],
            [np.sin(agent_yaw_rad), np.cos(agent_yaw_rad), 0],
            [0, 0, 1],
        ])
        coords_offset.append(transform_points(agent_coords, world_offset_from_agent))
    
    future_coords_offsets_pd.append(np.stack(coords_offset))
    timestamps.append(data["timestamp"].numpy().copy())
    agent_ids.append(data["track_id"].numpy().copy())

  0%|          | 0/5927 [00:00<?, ?it/s]C:\Users\Omar\Anaconda3\lib\site-packages\l5kit\dataset\agent.py:115: RuntimeWarning: disable_traffic_light_faces not found in config, this will raise an error in the future
  return self.get_frame(scene_index, state_index, track_id=track_id)
 28%|██▊       | 1665/5927 [51:00<7:29:58,  6.33s/it] 

In [14]:
write_pred_csv("submission/submission.csv",
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
              )